In [49]:
import psycopg2
import pandas as pd

# Connect to the PostgreSQL database
cnx = psycopg2.connect(user="archivai", password="Saad@2356925", host="archivai-database.postgres.database.azure.com", port=5432, database="postgres")
print("Connected successfully!")
cur = cnx.cursor()
query = """SELECT * FROM public."Page"
            ORDER BY "Id" ASC"""
cur.execute(query)
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=colnames)
df.head()

Connected successfully!


,Id,Language,MarkdownText,RawText,FileId,VersionId,SearchVector
0,43,english,"### Key Partner\n\n- AI/ML platforms (e.g., Az...","Key Partner\nAI/ML platforms (e.g., Azure ML)....",93,None,'acquisit':228 'activ':22 'advertis':131 'ai':...
1,66,english,"### Key Partner \nAI/ML platforms (e.g., Azur...","Key Partner AI/ML platforms (e.g., Azure ML). ...",102,None,'acquisit':228 'activ':22 'advertis':131 'ai':...
2,72,english,### Sex Ratio of Bronchial Carcinoma\n\n(1969)...,SEX RATIO OF BRONCHIAL CARCINOMA\n(1969) found...,110,None,'10':136 '15':62 '1959':64 '1963':218 '1969':6...
3,73,english,Connect,Connect,111,None,'connect':1
4,77,english,# Diagram Overview\n\n## Section\n- Name: stri...,Diagram Overview\n\nSection\n- Name: string\n-...,118,None,"'0':180 '1':173,182 '100':21 '10000000000':137..."


In [16]:
df.columns

Index(['Id', 'Language', 'MarkdownText', 'RawText', 'FileId', 'VersionId',
       'SearchVector'],
      dtype='object')

In [17]:
df.shape

(24, 7)

In [18]:
df['FileId'].nunique()

16

In [20]:
new_df = df[['FileId', 'RawText']]
new_df.head()

,FileId,RawText
0,93,"Key Partner\nAI/ML platforms (e.g., Azure ML)...."
1,102,"Key Partner AI/ML platforms (e.g., Azure ML). ..."
2,110,SEX RATIO OF BRONCHIAL CARCINOMA\n(1969) found...
3,111,Connect
4,118,Diagram Overview\n\nSection\n- Name: string\n-...


In [27]:
def get_file_content(df, file_id, file_id_col='FileId', raw_text_col='RawText'):
    """
    Get concatenated content for a specific file ID
    
    Parameters:
    df (DataFrame): DataFrame containing FileId and RawText columns
    file_id: The file ID to get content for
    file_id_col (str): Name of the FileId column
    raw_text_col (str): Name of the RawText column
    
    Returns:
    str: Concatenated raw text for the file ID
    """
    file_data = df[df[file_id_col] == file_id]
    if not file_data.empty:
        return '\n'.join(file_data[raw_text_col].astype(str))
    else:
        return f"No content found for FileId: {file_id}"


In [31]:
files_content = []
lables = []
for id in new_df['FileId'].unique():
    content = get_file_content(new_df, id)
    files_content.append(content)
    lables.append(str(id))
    
print("Files content and labels prepared.")
print(files_content[:5])  # Display first 5 contents
print(lables[:5])  # Display first 5 labels

Files content and labels prepared.
['Key Partner\nAI/ML platforms (e.g., Azure ML).\nCloud storage providers (e.g., Google Cloud, AWS).\nMarketing & Distribution partners.\nERP systems APIs.\nKey Activities\nProduct Development (Website and Mobile app).\nDeveloping and training AI models for document classification.\nContinuous improvement of the classification algorithm.\nData management and privacy.\nCustomer Support.\nValue Proposition\nAutomated document classification eliminates manual sorting and tagging.\nAI-enhanced document querying makes finding information easier.\nSimple and intuitive interface for offices and employees to manage their files.\nScalability: Can handle increasing amounts of documents without the need for more manual work.\nCustomer Relationships\nCustomer Support: Providing setup assistance and ongoing help through live chat and email.\nRegular updates: Providing new features, security patches, and improvements\nPersonalized service: Option to customize the A

In [29]:
files_content

['Key Partner\nAI/ML platforms (e.g., Azure ML).\nCloud storage providers (e.g., Google Cloud, AWS).\nMarketing & Distribution partners.\nERP systems APIs.\nKey Activities\nProduct Development (Website and Mobile app).\nDeveloping and training AI models for document classification.\nContinuous improvement of the classification algorithm.\nData management and privacy.\nCustomer Support.\nValue Proposition\nAutomated document classification eliminates manual sorting and tagging.\nAI-enhanced document querying makes finding information easier.\nSimple and intuitive interface for offices and employees to manage their files.\nScalability: Can handle increasing amounts of documents without the need for more manual work.\nCustomer Relationships\nCustomer Support: Providing setup assistance and ongoing help through live chat and email.\nRegular updates: Providing new features, security patches, and improvements\nPersonalized service: Option to customize the AI model for specific business needs

In [35]:
import sys
# Add the project root to Python path
sys.path.append('D:\Archivai-AI')

In [ ]:
import cohere
import os
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
from torch.utils.data import DataLoader, TensorDataset
from app.src.utils.classifier import TextClassifier
from app.src.utils.trainer import Trainer
from typing import List
from app.src.utils.config import encoder_path
import numpy as np

def get_file_content(df, file_id, file_id_col='FileId', raw_text_col='RawText'):
    """
    Get concatenated content for a specific file ID
    
    Parameters:
    df (DataFrame): DataFrame containing FileId and RawText columns
    file_id: The file ID to get content for
    file_id_col (str): Name of the FileId column
    raw_text_col (str): Name of the RawText column
    
    Returns:
    str: Concatenated raw text for the file ID
    """
    file_data = df[df[file_id_col] == file_id]
    if not file_data.empty:
        return '\n'.join(file_data[raw_text_col].astype(str))
    else:
        return f"No content found for FileId: {file_id}"

def train_model(chere_client: cohere.Client):
    # Connect to the PostgreSQL database
    cnx = psycopg2.connect(user="archivai", password="Saad@2356925", host="archivai-database.postgres.database.azure.com", port=5432, database="postgres")
    print("Connected successfully!")
    cur = cnx.cursor()
    query = """SELECT * FROM public."Page"
                ORDER BY "Id" ASC"""
    cur.execute(query)
    rows = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows, columns=colnames)

    # Slicing the DataFrame to get only FileId and RawText columns
    df_sliced = df[['FileId', 'RawText']]

    # Getting the content for each file
    files_content = []
    lables = []
    for id in new_df['FileId'].unique():
        content = get_file_content(df_sliced, id)
        files_content.append(content)
        lables.append(str(id))
    
    # get the embeddings
    raw_text = files_content
    embeddings = chere_client.embed(input_type= 'classification', texts= raw_text).embeddings

    # encoding the target variable 
    encoder = LabelEncoder()
    labels_encoded = encoder.fit_transform(lables)

    # Save the trained encoder
    joblib.dump(encoder, encoder_path)

    # converting to torch tensor
    embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)
    labels_tensor = torch.tensor(labels_encoded)

    # splitting the data to train and test
    embeddings_train, embeddings_test, classes_train, classes_test = train_test_split(
                embeddings_tensor, labels_tensor, test_size=0.20, random_state=0)
    
    # Create TensorDataset
    train_dataset = TensorDataset(embeddings_train, classes_train)
    test_dataset = TensorDataset(embeddings_test, classes_test)

    # Create DataLoaders
    batch_size = 32  
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
    # inistantiating a model
    model = TextClassifier(num_classes = len(labels))

    # choosing device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # calculating counts
    class_counts = np.bincount(labels_encoded)
    weights = 1.0 / torch.tensor(class_counts, dtype=torch.float32)
    weights = weights / weights.sum()

    # setting the criterion, optimizer and schedular
    criterion = torch.nn.CrossEntropyLoss(weight=weights.to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    # training and validating the model
    trainer_inist = Trainer(model, criterion, optimizer, scheduler, device)
    trainer_inist.train(train_loader, test_loader, num_epochs=30, patience= 5)

            
            
    

In [47]:
#from app.src.utils.building_model import train_model

# Train the model with the prepared data
train_model(raw_text=files_content, labels=lables)

c:\Users\laptop\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/30]
Training - Loss: 3.0006, Accuracy: 8.33%
Validation - Loss: 2.8019, Accuracy: 0.00%
Epoch [2/30]
Training - Loss: 2.3371, Accuracy: 41.67%
Validation - Loss: 2.8049, Accuracy: 0.00%
Epoch [3/30]
Training - Loss: 2.7797, Accuracy: 16.67%
Validation - Loss: 2.8114, Accuracy: 0.00%
Epoch [4/30]
Training - Loss: 2.5068, Accuracy: 8.33%
Validation - Loss: 2.8152, Accuracy: 0.00%
Epoch [5/30]
Training - Loss: 2.2438, Accuracy: 41.67%
Validation - Loss: 2.8192, Accuracy: 0.00%
Epoch [6/30]
Training - Loss: 2.1993, Accuracy: 25.00%
Validation - Loss: 2.8199, Accuracy: 0.00%
Early stopping triggered!
